In [2]:
import pandas as pd
import numpy
import os

Data Preprocess

In [3]:
def readData(path, languages, attribute):
    
    lyrics_data = pd.read_csv(path)
    lyrics_data = lyrics_data.dropna()
    lyrics_data = lyrics_data[lyrics_data.language.isin(languages)]
    lyrics = lyrics_data[attribute].values

    return lyrics

In [5]:
lyrics = readData('/Applications/ML projects/Song Lyrics/Dataset - 1/lyrics-data.csv', ['en'], 'Lyric')

In [7]:
def tokenization(lyrics, max_words):
    hashmap1 = {}
    for lyric in lyrics:
        for word in lyric:
            if word not in hashmap1:
                hashmap1[word] = 1
            else:
                hashmap1[word] += 1

    max_frequency = max(hashmap1.values()) + 1
    ranks = []
    hashmap2 = {}
    for key in hashmap1.keys():
        rank = max_frequency - hashmap1[key]
        hashmap2[key] = rank
        ranks.append(rank)
    ranks = sorted(ranks)
    threshold_rank = ranks[max_words]

    hashmap2Keys = hashmap2.keys()
    for key in hashmap2Keys:
        if hashmap2[key] >= threshold_rank:
            ranks.remove(hashmap2[key])
            hashmap2[key] = -1
    ranks = sorted(ranks)

    tokenizer = {}
    for key in hashmap2.keys():
        if hashmap2[key] != -1:
            rank = ranks.index(hashmap2[key]) + 1
            tokenizer[key] = rank
        
    tokenized_lyrics = []
    for lyric in lyrics:
        temp = []
        for word in lyric:
            if word in tokenizer:
                temp.append(tokenizer[word])
        tokenized_lyrics.append(temp)

    return tokenized_lyrics, tokenizer

In [8]:
def normalization(tokenized_lyrics, tokenizer):
    max_rank = float(max(tokenizer.values()))
    for i in range(len(tokenized_lyrics)):
        for j in range(len(tokenized_lyrics[i])):
            tokenized_lyrics[i][j] = tokenized_lyrics[i][j] / max_rank

In [9]:
def preprocess(lyrics, max_words, strip):
    split_lyrics = []

    for lyric in lyrics:
        split_lyrics.append(lyric.split(' '))

    tokenized_lyrics, tokenizer = tokenization(split_lyrics, max_words)
    normalization(tokenized_lyrics, tokenizer)
    
    normalized_lyrics = sorted(tokenized_lyrics, key=len)
    length = len(normalized_lyrics)
    lower_strip = int(strip * length)
    higher_strip = length - lower_strip
    LYRICS = normalized_lyrics[lower_strip: higher_strip]

    return LYRICS, tokenizer

In [10]:
LYRICS, TOKENIZER = preprocess(lyrics, 5000, 0.2)

KeyboardInterrupt: 